In [ ]:
#Импорт библиотек 
import numpy as np
import pandas as pd
from google.colab import drive
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Датафрейм с клиентами (почта, дата регистрации, текущий статус, дата смены статуса)

In [ ]:
#Считываем изначальные таблицы из Mailchimpa
df_sub=pd.read_csv("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/subscribed.csv")
df_unsub=pd.read_csv("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/unsubscribed.csv")
df_clean=pd.read_csv("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/cleaned.csv")

In [ ]:
#Соединение таблиц и корректировка объединенной 
df_sub.insert(3, 'STATUS_CHANGE', ' ' )
df_sub.insert(1, 'STATUS', 'subscribed' )
df_sub = df_sub[['Email Address', 'STATUS', 'STATUS_CHANGE', 'OPTIN_TIME']]
df_clean.insert(1, 'STATUS', 'cleaned' )
df_clean = df_clean[['Email Address', 'STATUS', 'CLEAN_TIME', 'OPTIN_TIME']]
df_clean = df_clean.rename(columns = {'CLEAN_TIME': 'STATUS_CHANGE'})
df_unsub.insert(1, 'STATUS', 'unsubscribed')
df_unsub = df_unsub[['Email Address', 'STATUS', 'UNSUB_TIME', 'OPTIN_TIME']]
df_unsub = df_unsub.rename(columns = {'UNSUB_TIME': 'STATUS_CHANGE'})
df = df_unsub.append(df_sub)
data = df_clean.append(df)
data['Email Address'] = data['Email Address'].str.lower()

In [ ]:
data.to_excel("DataFrame#1.xlsx")

# Sent

In [ ]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe

In [ ]:
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
auth.authenticate_user()#еще раз получаем доступ к диску
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
Campaign_name = gc.open('Кодификатор кампаний').worksheet('Summary')

In [ ]:
#Считывание гугл-таблицы со списком кампаний 2020-2021 гг.
campaigns = pd.DataFrame(Campaign_name.get_all_records())
campaigns = campaigns[['Дата','Категория','Кампания ']]

In [ ]:
#Считывание всех 990 файлов из папки
url = '/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Sent/'
df_list=[]
for j, campaign in campaigns.iterrows(): 
    suffics=".csv"
    url_1=url+campaign['Кампания ']+suffics
    interim=pd.read_csv(url_1) 
    interim['Email Address']=interim['Email Address'].str.lower()
    interim['Campaign']=campaign['Кампания ']
    interim = interim[['Email Address', 'Campaign']]
    df_list.append(interim)

In [ ]:
sent_df=pd.concat(df_list)

In [ ]:
sent_df.to_csv('Sent.csv')

# Opened

In [ ]:
import os
url = "/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Opened/"
filenames=[]
for filename in os.walk(url):
    filenames=filename[2]

In [ ]:
#Использование библиотеки os для считывания файлов opened 
df_opened_list=[]
list_of_errors_opened=[]
for i, campaign in enumerate(filenames): 
    print(i)
    url_1=url+campaign
    try:
      interim=pd.read_csv(url_1) 
      interim['Email Address']=interim['Email Address'].str.lower()
      interim['Campaign']=campaign.replace('.csv', "")
      interim = interim[['Email Address', 'Campaign']]
      df_opened_list.append(interim)
    except KeyError:
      list_of_errors_opened.append(campaign)

In [ ]:
opened_df=pd.concat(df_opened_list)

In [ ]:
#Добавление эксель-таблиц с остальной информацией по статусам 
xl_1 = pd.read_excel('/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Онлайн_живопись.xlsx')
xl_2 = pd.read_excel('/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Онлайн_живопись2.xlsx')
xl_3 = pd.read_excel('/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Онлайн_живопись3.xlsx')
xl_1 = xl_1.rename(columns = {'E-mail': 'Email Address', 'Кампания': 'Campaign'})
xl_2 = xl_2.rename(columns = {'E-mail': 'Email Address', 'Кампания': 'Campaign'})
xl_3 = xl_3.rename(columns = {'E-mail': 'Email Address', 'Кампания': 'Campaign'})

In [ ]:
xl1_opened = xl_1[xl_1['Статус'] == 'Opened']
xl1_unsub = xl_1[xl_1['Статус'] == 'Unsubscribed']
xl1_clicked = xl_1[xl_1['Статус'] == 'Clicked']
xl2_opened = xl_2[xl_2['Статус'] == 'Opened']
xl2_unsub = xl_2[xl_2['Статус'] == 'Unsubscribed']
xl2_clicked = xl_2[xl_2['Статус'] == 'Clicked']
xl3_opened = xl_3[xl_3['Статус'] == 'Opened']
xl3_unsub = xl_3[xl_3['Статус'] == 'Unsubscribed']
xl3_clicked = xl_3[xl_3['Статус'] == 'Clicked']
opened_xlsx = (xl1_opened.append(xl2_opened)).append(xl3_opened)
opened_xlsx = opened_xlsx[['Email Address', 'Campaign']]

In [ ]:
opened_df = opened_df.append(opened_xlsx)

In [ ]:
opened_df.to_csv('Opened.csv')

# Bounced

In [ ]:
url = "/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Bounced/"
filenames_1=[]
for filename in os.walk(url):
    filenames_1=filename[2]

In [ ]:
df_bounced_list=[]
list_of_errors_bounced=[]
for i, campaign in enumerate(filenames_1): 
    print(i)
    url_1=url+campaign
    try:
      interim=pd.read_csv(url_1) 
      interim['Email Address']=interim['Email Address'].str.lower()
      interim['Campaign']=campaign.replace('.csv', "")
      interim = interim[['Email Address', 'Campaign']]
      df_bounced_list.append(interim)
    except KeyError:
      list_of_errors_opened.append(campaign)

In [ ]:
bounced_df=pd.concat(df_bounced_list)

In [ ]:
bounced_df.to_csv('Bounced.csv')

# Clicked

In [ ]:
clicked_xlsx = (xl1_clicked.append(xl2_clicked)).append(xl3_clicked)
clicked_xlsx = clicked_xlsx[['Email Address', 'Campaign']]

In [ ]:
url = "/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Clicked/"
filenames_2=[]
for filename in os.walk(url):
    filenames_2=filename[2]

In [ ]:
df_clicked_list=[]
list_of_errors_clicked=[]
for i, campaign in enumerate(filenames_2): 
    print(i)
    url_1=url+campaign
    try:
      interim=pd.read_csv(url_1) 
      interim['Email Address']=interim['Email Address'].str.lower()
      interim['Campaign']=campaign.replace('.csv', "")
      interim = interim[['Email Address', 'Campaign']]
      df_clicked_list.append(interim)
    except KeyError:
      list_of_errors_clicked.append(campaign)

In [ ]:
clicked_df=pd.concat(df_clicked_list)

In [ ]:
clicked_df = clicked_df.append(clicked_xlsx)

In [ ]:
clicked_df.to_csv('Clicked.csv')

# Unsubscribed

In [ ]:
unsub_xlsx = (xl1_unsub.append(xl2_unsub)).append(xl3_unsub)
unsub_xlsx = unsub_xlsx[['Email Address', 'Campaign']]

In [ ]:
url = "/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Clicked/"
filenames_3=[]
for filename in os.walk(url):
    filenames_3=filename[2]

In [ ]:
df_unsub_list=[]
list_of_errors_unsub=[]
for i, campaign in enumerate(filenames_3): 
    print(i)
    url_1=url+campaign
    try:
      interim=pd.read_csv(url_1) 
      interim['Email Address']=interim['Email Address'].str.lower()
      interim['Campaign']=campaign.replace('.csv', "")
      interim = interim[['Email Address', 'Campaign']]
      df_unsub_list.append(interim)
    except KeyError:
      list_of_errors_unsub.append(campaign)

In [ ]:
unsub_df=pd.concat(df_unsub_list)

In [ ]:
unsub_df = unsub_df.append(unsub_xlsx)

In [ ]:
unsub_df.to_csv('Unsubscribed.csv')

# Orders

In [ ]:
#Соединение датафрейма с кодификатором кампаний и с данными по открытиям писем 
opened = pd.read_csv("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/Opened.csv")
campaigns = campaigns.rename(columns = {'Кампания ': 'Campaign'})
data = pd.read_excel("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/DataFrame#1.xlsx")
opened_date = pd.merge(campaigns, opened, left_on = 'Campaign', right_on = 'Campaign')

In [ ]:
%%bigquery --project basic-bounty-272217 orders
SELECT
  useremail as Email,
  pay_month as date_of_pay,
  COUNT(DISTINCT(order_index)) as number_of_orders,
  SUM(pricepaid) as revenue
FROM `metrics_combined.metrics_cohort`
GROUP BY useremail, pay_month

In [ ]:
#Вывод заказов из базы 
orders0 = orders.rename(columns = {'Email': 'Email Address'})
orders0

,Email Address,date_of_pay,number_of_orders,revenue
0,None,None,0,NaN
1,lapolga54@yandex.ru,2019-02-01,1,900.0
2,kuzmich.svetlana@gmail.com,2019-02-01,1,900.0
3,arepchanskaya@yandex.ru,2019-02-01,1,900.0
4,loto80@yandex.ru,2019-02-01,1,900.0
...,...,...,...,...
131323,shymanskayaaa@gmail.com,2021-03-01,1,0.0
131324,checheneva@amet.ru,2021-03-01,1,0.0
131325,natalie.kotova@yandex.ru,2021-03-01,1,0.0
131326,omishura@yandex.ru,2021-03-01,1,0.0


In [ ]:
#Объединение файла с заказами, изначальными данными по добавлению пользователей в базу
opened_and_orders = pd.merge(orders0, opened_date, left_on = 'Email Address', right_on = 'Email Address')
opened_and_orders = pd.merge(opened_and_orders, data, left_on = 'Email Address', right_on = 'Email Address')

In [ ]:
#Подготовление данных для вывода временного лага между датой оплаты заказа и датой отправки письма 
opened_and_orders['Дата'] = pd.to_datetime(opened_and_orders['Дата'])
opened_and_orders['date_of_pay'] = pd.to_datetime(opened_and_orders['date_of_pay'])
a = opened_and_orders['date_of_pay']
b = opened_and_orders['Дата']
c = a - b.shift(0)
opened_and_orders.insert(3, 'DATEDIF', c)

In [ ]:
#Фильтрация заказов по разнице во времени 
from pandas import to_timedelta
opened_and_orders = opened_and_orders[(opened_and_orders['DATEDIF'] <= to_timedelta(6, unit = "D")) & (opened_and_orders['DATEDIF'] >= to_timedelta(0, unit = "D"))]
opened_and_orders = opened_and_orders[['Email Address', 'date_of_pay', 'OPTIN_TIME', 'Дата', 'DATEDIF', 'number_of_orders', 'revenue']]
opened_and_orders = opened_and_orders.sort_values(by = ['Email Address'], ascending = True) 
opened_and_orders

,Email Address,date_of_pay,OPTIN_TIME,Дата,DATEDIF,number_of_orders,revenue
1523323,00eurika00@mail.ru,2020-04-01,2020-03-30 01:34:01,2020-03-31,1 days,1,900.0
1292767,0104olga@mail.ru,2020-04-01,2020-02-10 12:35:35,2020-03-30,2 days,1,620.0
1292766,0104olga@mail.ru,2020-04-01,2020-02-10 12:35:35,2020-03-31,1 days,1,620.0
547748,01gerasu@gmail.com,2022-02-01,2021-12-23 13:58:39,2022-01-26,6 days,1,0.0
1409035,05sveta2018@gmail.com,2020-03-01,2020-01-09 12:48:51,2020-02-25,5 days,1,3100.0
...,...,...,...,...,...,...,...
1631457,zynovieva67@mail.ru,2021-11-01,2021-10-08 10:56:12,2021-10-27,5 days,1,2800.0
1631455,zynovieva67@mail.ru,2021-11-01,2021-10-08 10:56:12,2021-10-30,2 days,1,2800.0
1435890,zz108108@gmail.com,2021-10-01,2021-03-31 17:06:59,2021-09-29,2 days,1,5040.0
1435889,zz108108@gmail.com,2021-10-01,2021-03-31 17:06:59,2021-09-30,1 days,1,5040.0


In [ ]:
#Агрегация по повторным отправкам писем 
opened_and_orders0 = opened_and_orders.groupby(['Email Address', 'Дата']).agg('sum')
opened_and_orders0 = (opened_and_orders0.sort_values(by = ['Email Address'], ascending = True)).reset_index()
opened_and_orders0

,Email Address,Дата,number_of_orders,revenue
0,00eurika00@mail.ru,2020-03-31,1,900.0
1,0104olga@mail.ru,2020-03-30,1,620.0
2,0104olga@mail.ru,2020-03-31,1,620.0
3,01gerasu@gmail.com,2022-01-26,1,0.0
4,05sveta2018@gmail.com,2020-02-25,1,3100.0
...,...,...,...,...
18147,zynovieva67@mail.ru,2021-10-28,1,2800.0
18148,zynovieva67@mail.ru,2021-10-30,1,2800.0
18149,zz108108@gmail.com,2021-09-29,1,5040.0
18150,zz108108@gmail.com,2021-07-29,3,31120.0


In [ ]:
#Вывод общей таблицы по заказам со поправками
opened_and_orders = pd.merge(opened_and_orders0, opened_and_orders, how = 'inner', left_on = ['Email Address', 'Дата'], right_on = ['Email Address', 'Дата'])
opened_and_orders = opened_and_orders.rename(columns = {'number_of_orders_x': 'number_of_orders', 'revenue_x': 'revenue'})
opened_and_orders = opened_and_orders[['Email Address', 'OPTIN_TIME', 'Дата', 'date_of_pay', 'DATEDIF', 'number_of_orders', 'revenue']]

In [ ]:
#Вывод таблицы по минимальному временному лагу 
opened_and_orders['DATEDIF'] = pd.to_numeric(opened_and_orders['DATEDIF'])
opened_and_orders_min = opened_and_orders.groupby('Email Address').apply(lambda x: min(list(x.DATEDIF)))
opened_and_orders_min = opened_and_orders_min.reset_index()
opened_and_orders_min = opened_and_orders_min.rename(columns = {0: 'DATEDIF'})
opened_and_orders_min

,Email Address,DATEDIF
0,00eurika00@mail.ru,86400000000000
1,0104olga@mail.ru,86400000000000
2,01gerasu@gmail.com,518400000000000
3,05sveta2018@gmail.com,172800000000000
4,070708pnm@gmail.com,86400000000000
...,...,...
8722,zvivanova@beeline.ru,345600000000000
8723,zvonariova11@gmail.com,86400000000000
8724,zwing78@yandex.ru,432000000000000
8725,zynovieva67@mail.ru,172800000000000


In [ ]:
#Итоговая таблица по заказам с минимумами 
opened_and_orders = pd.merge(opened_and_orders, opened_and_orders_min, how = 'inner', left_on = ['Email Address', 'DATEDIF'], right_on = ['Email Address', 'DATEDIF'])
opened_and_orders.head(20)

,Email Address,OPTIN_TIME,Дата,date_of_pay,DATEDIF,number_of_orders,revenue
0,01gerasu@gmail.com,2021-12,2022-01-26,2022-02,518400000000000,1,0.0
1,070708pnm@gmail.com,2021-04,2021-11-30,2021-12,86400000000000,1,6000.0
2,081a6z@gmail.com,2021-02,2021-11-30,2021-12,86400000000000,1,2800.0
3,1000_pismo@list.ru,2021-11,2021-11-01,2021-11,0,3,4300.0
4,1111707@mail.ru,2021-01,2021-02-01,2021-02,0,1,2800.0
5,1205851@mail.ru,2021-09,2021-09-30,2021-10,86400000000000,1,2480.0
6,12kova@mail.ru,2021-02,2021-04-01,2021-04,0,1,3100.0
7,1442266@mail.ru,2021-06,2021-11-01,2021-11,0,1,2325.0
8,191640@gmail.com,2021-11,2021-11-25,2021-12,518400000000000,2,4080.0
9,1921577@gmail.com,2021-05,2021-05-30,2021-06,172800000000000,1,2480.0


In [ ]:
opened_and_orders['OPTIN_TIME'] = pd.to_datetime(opened_and_orders['OPTIN_TIME'], errors = 'coerce').dt.floor('d')
opened_and_orders['date_of_pay'] = pd.to_datetime(opened_and_orders['date_of_pay'])
opened_and_orders = opened_and_orders.drop_duplicates()

In [ ]:
#Сводная таблица когорты(год-месяц)-заказы-выручка
opened_and_orders['OPTIN_TIME'] = opened_and_orders['OPTIN_TIME'].dt.strftime("%Y-%m")
opened_and_orders = opened_and_orders[opened_and_orders['OPTIN_TIME'] >= '2021-01']
opened_and_orders['date_of_pay'] = opened_and_orders['date_of_pay'].dt.strftime("%Y-%m")
orders = opened_and_orders.pivot_table(['number_of_orders'], ['OPTIN_TIME', 'date_of_pay'], aggfunc = 'sum')
revenue = opened_and_orders.pivot_table(['revenue'], ['OPTIN_TIME', 'date_of_pay'], aggfunc = 'sum')

In [ ]:
opened_and_orders[opened_and_orders['date_of_pay'] == '2021-11']

,Email Address,OPTIN_TIME,Дата,date_of_pay,DATEDIF,number_of_orders,revenue
3,1000_pismo@list.ru,2021-11,2021-11-01,2021-11,0,3,4300.0
7,1442266@mail.ru,2021-06,2021-11-01,2021-11,0,1,2325.0
10,1984scorpion@rambler.ru,2021-07,2021-11-01,2021-11,0,4,29700.0
18,2728183@mail.ru,2021-03,2021-11-01,2021-11,0,4,12400.0
22,2peterivanov@gmail.com,2021-11,2021-10-28,2021-11,345600000000000,1,5250.0
...,...,...,...,...,...,...,...
3994,zlobina.os@gmail.com,2021-06,2021-11-01,2021-11,0,4,12480.0
4001,zorinanatalia@gmail.com,2021-10,2021-11-01,2021-11,0,2,6200.0
4005,zulia0116@gmail.com,2021-10,2021-11-01,2021-11,0,4,20400.0
4009,zumareva@mail.ru,2021-06,2021-11-01,2021-11,0,2,36000.0


In [ ]:
orders = orders['number_of_orders'].unstack(1).fillna(0)
revenue = revenue['revenue'].unstack(1).fillna(0)

In [ ]:
orders = orders.reset_index()
revenue = revenue.reset_index()

In [ ]:
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
table0 = gc.open("Когортный анализ_заказы с писем").worksheet('Заказы')
set_with_dataframe(table0, orders)
table1 = gc.open("Когортный анализ_заказы с писем").worksheet('Выручка')
set_with_dataframe(table1, revenue)

In [ ]:
#Сводная таблица когорта(год-неделя)-заказы-выручка
opened_and_orders['OPTIN_TIME'] = opened_and_orders['OPTIN_TIME'].dt.strftime("%Y-%W")
opened_and_orders_pivot_table2 = opened_and_orders.pivot_table(['revenue', 'number_of_orders'], ['OPTIN_TIME'], aggfunc = 'sum')
opened_and_orders_pivot_table2.to_excel('Pivot_orders(weeks).xlsx')

AttributeError: ignored

# Итоговая таблица с метриками 

In [ ]:
#Считывание сводных таблиц по статусам и заказам 
pivot_sent = pd.read_excel("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/Pivot_sent_w.xlsx")
pivot_opened = pd.read_excel("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/Pivot_opened_w.xlsx")
pivot_unsub = pd.read_excel("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/Pivot_unsub_w.xlsx")
pivot_clicked = pd.read_excel("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/Pivot_clicked_w.xlsx")
pivot_bounced = pd.read_excel("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/Pivot_bounced_w.xlsx")
pivot_orders = pd.read_excel("/content/drive/MyDrive/Синхронизация/Аналитика/Retention/Mailchimp/Dashboard/Base/Pivot_orders(weeks).xlsx")

In [ ]:
#Создание сводной таблицы по когортам изначальной базы 
data['OPTIN_TIME'] = pd.to_datetime(data['OPTIN_TIME'], errors = 'coerce').dt.floor('d')
data['OPTIN_TIME'] = data['OPTIN_TIME'].dt.strftime("%Y-%W")
pivot_data = data.pivot_table(['Email Address'], ['OPTIN_TIME'], aggfunc = 'count')
pivot_data = pivot_data.reset_index()
pivot_data

,OPTIN_TIME,Email Address
0,2018-47,56
1,2018-48,1576
2,2018-49,4235
3,2018-50,295
4,2018-51,96
...,...,...
141,2022-01,1229
142,2022-02,1160
143,2022-03,853
144,2022-04,1279


In [ ]:
#Создание таблицы с метриками 
metrics = pd.DataFrame(columns = ['cohort', 'size', 'OR', 'CR', 'CTOR', 'Orders', 'Revenue', 'Unsubscribed', 'Bounced'])

In [ ]:
#Соединение всех сводных таблиц 
pivot_sent = pivot_sent[(pivot_sent['OPTIN_TIME'] > '2019-52') & (pivot_sent['OPTIN_TIME'] < '2022-04')]
pivot_opened = pivot_opened[(pivot_opened['OPTIN_TIME'] > '2019-52') & (pivot_opened['OPTIN_TIME'] < '2022-04')]
pivot_clicked = pivot_clicked[(pivot_clicked['OPTIN_TIME'] > '2019-52') & (pivot_clicked['OPTIN_TIME'] < '2022-04')]
pivot_unsub = pivot_unsub[(pivot_unsub['OPTIN_TIME'] > '2019-52') & (pivot_unsub['OPTIN_TIME'] < '2022-04')]
pivot_data = pivot_data[(pivot_data['OPTIN_TIME'] > '2019-52') & (pivot_data['OPTIN_TIME'] < '2022-04')]
pivot_bounced = pivot_bounced[(pivot_bounced['OPTIN_TIME'] > '2019-52') & (pivot_bounced['OPTIN_TIME'] < '2022-04')]
pivot_sent = pivot_sent.rename(columns = {'Email Address': 'count_sent'})
pivot_opened = pivot_opened.rename(columns = {'Email Address': 'count_opened'})
pivot_clicked = pivot_clicked.rename(columns = {'Email Address': 'count_clicked'})
pivot_unsub = pivot_unsub.rename(columns = {'Email Address': 'count_unsub'})
pivots = pd.merge(pivot_data, pivot_sent, left_on = 'OPTIN_TIME', right_on = 'OPTIN_TIME')
pivots = pd.merge(pivots, pivot_unsub, left_on = 'OPTIN_TIME', right_on = 'OPTIN_TIME')
pivots = pd.merge(pivots, pivot_bounced, left_on = 'OPTIN_TIME', right_on = 'OPTIN_TIME')
pivots = pd.merge(pivots, pivot_opened, left_on = 'OPTIN_TIME', right_on = 'OPTIN_TIME')
pivots = pd.merge(pivots, pivot_clicked, left_on = 'OPTIN_TIME', right_on = 'OPTIN_TIME')
pivots = pd.merge(pivots, pivot_orders, left_on = 'OPTIN_TIME', right_on = 'OPTIN_TIME')
pivots

,OPTIN_TIME,Email Address_x,count_sent,count_unsub,Email Address_y,count_opened,count_clicked,number_of_orders,revenue
0,2020-01,1052,251150,697,94,35664,2979,215.0,862015.0
1,2020-02,386,94210,229,13,13716,1248,72.0,360590.0
2,2020-03,211,50531,156,17,8498,956,77.0,586876.0
3,2020-04,370,87298,251,31,11864,1136,89.0,340827.0
4,2020-05,247,57307,148,19,9349,878,49.0,185201.0
...,...,...,...,...,...,...,...,...,...
102,2021-51,704,6563,8,27,5269,724,23.0,66289.0
103,2022-00,422,2685,8,8,2213,303,6.0,23870.0
104,2022-01,1229,7315,28,20,6047,510,16.0,65355.0
105,2022-02,1160,5048,12,71,4148,345,23.0,91845.0


In [ ]:
#Расчет основных метрик 
metrics['OR'] = pivots['count_opened']/pivots['count_sent']
metrics['CR'] = pivots['count_clicked']/pivots['count_sent']
metrics['CTOR'] = pivots['count_clicked']/pivots['count_opened']
metrics['Orders'] = pivots['number_of_orders']
metrics['Revenue'] = pivots['revenue']
metrics['cohort'] = pivots['OPTIN_TIME']
metrics['size'] = pivots['Email Address_x']
metrics['Unsubscribed'] = pivots['count_unsub']/pivots['Email Address_x']
metrics['Bounced'] = pivots['Email Address_y']/pivots['Email Address_x']

In [ ]:
#Общая таблица с метриками по когортам год-неделя 
metrics

,cohort,size,OR,CR,CTOR,Orders,Revenue,Unsubscribed,Bounced
0,2020-01,1052,0.142003,0.011861,0.083530,215.0,862015.0,0.662548,0.089354
1,2020-02,386,0.145590,0.013247,0.090989,72.0,360590.0,0.593264,0.033679
2,2020-03,211,0.168174,0.018919,0.112497,77.0,586876.0,0.739336,0.080569
3,2020-04,370,0.135902,0.013013,0.095752,89.0,340827.0,0.678378,0.083784
4,2020-05,247,0.163139,0.015321,0.093914,49.0,185201.0,0.599190,0.076923
...,...,...,...,...,...,...,...,...,...
102,2021-51,704,0.802834,0.110315,0.137407,23.0,66289.0,0.011364,0.038352
103,2022-00,422,0.824209,0.112849,0.136918,6.0,23870.0,0.018957,0.018957
104,2022-01,1229,0.826658,0.069720,0.084339,16.0,65355.0,0.022783,0.016273
105,2022-02,1160,0.821712,0.068344,0.083173,23.0,91845.0,0.010345,0.061207


In [ ]:
pivot_data.to_excel('Pivot_data.xlsx')

In [ ]:
metrics.to_excel('metrics (weeks).xlsx')